## Dataset description

The dataset for Lab-2 is sample of the SBA dataset posted on Kaggle. 

The dataset is from the U.S. Small Business Administration (SBA)
The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market 
(SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment.
There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. 
However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.

More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied


In [199]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import numpy as np

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## The function `describe_more` is a data exploration utility function


In [1]:
"""
Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:200].to_dict())
                
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

## Load data

We assume data is located in the folder `data` where we run our notebook

In [201]:
data = pd.read_csv('/Users/aishwaryaadiki/Downloads/SBA_loans_small(2).csv')
data.head()

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,CHGOFF,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,CHGOFF,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,CHGOFF,"$26,000.00","$23,400.00"


In [202]:
data.shape

(448583, 20)

In [203]:
desc_df = describe_more(data)

In [204]:
pd.set_option('display.width', 200)
desc_df.head(n=200)

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,24757,object,1,30,"{'LOS ANGELES': 5751, 'HOUSTON': 5102, 'NEW YO..."
1,State,51,object,2,3,"{'CA': 65183, 'TX': 35161, 'NY': 28716, 'FL': ..."
2,Zip,28185,int64,1,5,"{90015: 492, 10001: 455, 93401: 385, 90010: 37..."
3,Bank,5156,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 43347, 'WELLS F..."
4,BankState,54,object,2,3,"{'CA': 58768, 'NC': 39709, 'IL': 32982, 'OH': ..."
5,NAICS,1290,int64,1,6,"{0: 100776, 722110: 13888, 722211: 9713, 81111..."
6,Term,392,int64,1,3,"{84: 114519, 60: 44888, 240: 43020, 120: 38908..."
7,NoEmp,466,int64,1,4,"{1: 76992, 2: 68936, 3: 45269, 4: 36503, 5: 30..."
8,NewExist,3,float64,3,3,"{1.0: 321541, 2.0: 126457, 0.0: 520}"
9,CreateJob,196,int64,1,4,"{0: 313743, 1: 31439, 2: 29216, 3: 14422, 4: 1..."


## By looking at the levels columns, we can tell that there are only 2 possible valid values for MIS_Status 
## There are 369804 instances of 'P I F'
## There are 78779 instances of 'CHGOFF'

#### **Encoding target variable `MIS_Status` into `0` and `1`, when value `P I F` is mapped to `0` and `CHGOFF` is mapped to `1`.  Replacing values in the original column**

In [205]:
data.replace('CHGOFF', 1, inplace=True)
data.replace('P I F', 0, inplace=True)
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00"
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,92649,WELLS FARGO BANK NATL ASSOC,SD,0,300,10,1.0,0,0,1,0,N,N,"$492,000.00",$0.00,0,"$492,000.00","$344,400.00"
448579,DOVER,NH,3820,CITIZENS BANK NATL ASSOC,RI,812112,36,1,2.0,0,1,0,1,N,N,"$10,000.00",$0.00,0,"$10,000.00","$5,000.00"
448580,LUBBOCK,TX,79423,PLAINSCAPITAL BANK,TX,611691,64,1,2.0,1,1,52007,1,N,N,"$68,000.00",$0.00,0,"$68,000.00","$57,800.00"
448581,SULLIVAN,IL,61951,FIRST MID-ILLINOIS BK & TRUST,IL,0,60,2,1.0,0,0,1,0,N,Y,"$23,200.00",$0.00,0,"$23,200.00","$18,560.00"


## **Thinking about how to encode `ZIP` column...**

- Should I leave it as is? No, too many values with a huge range.
- Should I encode it using one-hot-encoding?
- Should I encode it using target encoder?

In [206]:
from sklearn.preprocessing import OneHotEncoder
data1 = pd.read_csv('/Users/aishwaryaadiki/Downloads/SBA_loans_small(2).csv')

enc = OneHotEncoder(handle_unknown='ignore')
encoder=enc.fit_transform(data1[['Zip']])
print(encoder)

  (0, 23041)	1.0
  (1, 27587)	1.0
  (2, 24076)	1.0
  (3, 21882)	1.0
  (4, 24582)	1.0
  (5, 12580)	1.0
  (6, 23599)	1.0
  (7, 13681)	1.0
  (8, 22447)	1.0
  (9, 24176)	1.0
  (10, 26781)	1.0
  (11, 4227)	1.0
  (12, 224)	1.0
  (13, 2729)	1.0
  (14, 23669)	1.0
  (15, 16156)	1.0
  (16, 22540)	1.0
  (17, 17903)	1.0
  (18, 15992)	1.0
  (19, 9634)	1.0
  (20, 6792)	1.0
  (21, 10766)	1.0
  (22, 25208)	1.0
  (23, 15963)	1.0
  (24, 11411)	1.0
  :	:
  (448558, 5132)	1.0
  (448559, 2311)	1.0
  (448560, 12670)	1.0
  (448561, 21844)	1.0
  (448562, 24154)	1.0
  (448563, 11232)	1.0
  (448564, 22380)	1.0
  (448565, 25133)	1.0
  (448566, 25587)	1.0
  (448567, 15822)	1.0
  (448568, 10031)	1.0
  (448569, 17149)	1.0
  (448570, 21903)	1.0
  (448571, 6004)	1.0
  (448572, 10318)	1.0
  (448573, 127)	1.0
  (448574, 9090)	1.0
  (448575, 2987)	1.0
  (448576, 26779)	1.0
  (448577, 25167)	1.0
  (448578, 25714)	1.0
  (448579, 1006)	1.0
  (448580, 22773)	1.0
  (448581, 17941)	1.0
  (448582, 3745)	1.0


## Since this takes up too much memory, it is not a good idea to use One-hot encoder


Encode ZIP column using target **(mean)** encoding technique.
At this point `MIS_Status` column contains only 0 and 1.
Replacing `ZIP` values with the mean of the response column (`MIS_Status`) for each value in a `ZIP` column. 

For example, if for ZIP=11222 there are total of 150 records. Out of which 80 records have label `0` and 70 records have label `1`.   

Populate new column value `ZIP_TE` for the records with `ZIP=11222` with value:  
(0x80 + 1x70)/(80+70) = 0.467


In [207]:
MIS_Enc = data[data["MIS_Status"] == 1].groupby('Zip').size()/data[data["MIS_Status"]== 0].groupby('Zip').size()
data.loc[:, 'Zip'] = data['Zip'].map(MIS_Enc)
data['Zip']

0         0.128205
1         0.088000
2         0.181818
3         0.328358
4         0.052239
            ...   
448578    0.142857
448579    0.089286
448580    0.087912
448581    0.125000
448582    0.125000
Name: Zip, Length: 448583, dtype: float64

**I SHOULD USE Target ENCODING INSTEAD OF ONE-HOT Encoding**

#### **Encoding `LowDoc` column using `from sklearn import preprocessing.LabelEncoder`. Replaced original column.**

In [224]:
from sklearn.preprocessing import LabelEncoder
data['LowDoc'] = LabelEncoder().fit_transform(data.NewExist)

data['LowDoc'].describe()

count    448583.000000
mean          1.281034
std           0.452396
min           0.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           3.000000
Name: LowDoc, dtype: float64

## Fill all columns with missing values with `Unknown` or equivalent value.

```
# filling with Unknown class
df_clean = df.fillna("Unknown")
df_clean
```

In [210]:
for col in data.columns:
    if data[col].isna().any() == True:
        print(col)

City
State
Zip
Bank
BankState
NewExist
RevLineCr


In [211]:
df_clean=data.fillna("Unknown")
df_clean

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,0.128205,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,1,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00"
1,SEATTLE,WA,0.088,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,1,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,0.181818,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,1,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00"
3,HOUSTON,TX,0.328358,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,1,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,0.052239,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,2,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,0.142857,WELLS FARGO BANK NATL ASSOC,SD,0,300,10,1.0,0,0,1,0,N,1,"$492,000.00",$0.00,0,"$492,000.00","$344,400.00"
448579,DOVER,NH,0.089286,CITIZENS BANK NATL ASSOC,RI,812112,36,1,2.0,0,1,0,1,N,2,"$10,000.00",$0.00,0,"$10,000.00","$5,000.00"
448580,LUBBOCK,TX,0.087912,PLAINSCAPITAL BANK,TX,611691,64,1,2.0,1,1,52007,1,N,2,"$68,000.00",$0.00,0,"$68,000.00","$57,800.00"
448581,SULLIVAN,IL,0.125,FIRST MID-ILLINOIS BK & TRUST,IL,0,60,2,1.0,0,0,1,0,N,1,"$23,200.00",$0.00,0,"$23,200.00","$18,560.00"


## **H20-DataFrame**

Load same `SBA_loans_small.zip` into H2O-3 dataframe.  
Describe imported dataframe.

In [212]:
import h2o 
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 days 1 hour 7 mins
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_aishwaryaadiki_pztdgv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.491 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [216]:
df_h=h2o.import_file('/Users/aishwaryaadiki/Downloads/SBA_loans_small(2).zip')
df_h.describe()
dataframe1 = df_h

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:448583
Cols:20

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
type,enum,enum,int,enum,enum,int,int,int,int,int,int,int,int,enum,enum,enum,int,enum,enum,enum
mins,,,0.0,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,,,
mean,,,53841.996368565,,,398694.68236201577,110.78555807955277,11.40656912990461,1.2799479166666672,8.606342638931922,10.9682333035358,2760.593203487426,0.7566314372145174,,,,0.0,,,
maxs,,,99999.0,,,928120.0,527.0,9999.0,2.0,8800.0,8800.0,99999.0,2.0,,,,0.0,,,
sigma,,,31169.478015031877,,,263264.9256337997,78.85420615377282,72.25402961839185,0.451514229420894,239.7101247911409,240.13097630419077,12783.22848127573,0.6458408354967026,,,,0.0,,,
zeros,,,139,,,100755,394,3318,513,313701,219984,103706,161424,,,,448581,,,
missing,13,6,0,755,757,0,0,0,71,0,0,0,0,130648,2037,0,2,0,0,0
0,BUFFALO,NY,14218.0,BANK OF AMERICA NATL ASSOC,NC,812111.0,2.0,7.0,1.0,0.0,0.0,1.0,1.0,Y,N,"$74,104.00",0.0,CHGOFF,"$16,000.00","$8,000.00"
1,WEST MIFFLIN,PA,15122.0,CITIZENS BANK NATL ASSOC,RI,236115.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,Y,N,"$76,955.00",0.0,CHGOFF,"$35,000.00","$17,500.00"
2,SAN DIEGO,CA,92126.0,MUFG UNION BANK NATL ASSOC,CA,442110.0,5.0,6.0,1.0,0.0,6.0,1.0,1.0,,N,"$24,985.00",0.0,CHGOFF,"$25,000.00","$12,500.00"


In [217]:
df_h.columns

['City',
 'State',
 'Zip',
 'Bank',
 'BankState',
 'NAICS',
 'Term',
 'NoEmp',
 'NewExist',
 'CreateJob',
 'RetainedJob',
 'FranchiseCode',
 'UrbanRural',
 'RevLineCr',
 'LowDoc',
 'DisbursementGross',
 'BalanceGross',
 'MIS_Status',
 'GrAppv',
 'SBA_Appv']

In [218]:
df_h['City'] = (df_h['City'].isna()).ifelse('Missing', df_h['City'])
df_h['State'] = (df_h['State'].isna()).ifelse('Missing', df_h['State'])
df_h['Zip'] = (df_h['Zip'].isna()).ifelse('Missing', df_h['Zip'])
df_h['Bank'] = (df_h['Bank'].isna()).ifelse('Missing', df_h['Bank'])
df_h['BankState'] = (df_h['BankState'].isna()).ifelse('Missing', df_h['BankState'])
df_h['NewExist'] = df_h['NewExist'].fillna(method="forward",axis=0,maxlen=1)
df_h['RevLineCr'] = (df_h['RevLineCr'].isna()).ifelse('Missing', df_h['RevLineCr'])
df_h['LowDoc'] = (df_h['LowDoc'].isna()).ifelse('Missing', df_h['LowDoc'])
df_h


City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
BUFFALO,NY,14218,BANK OF AMERICA NATL ASSOC,NC,812111,2,7,1,0,0,1,1,Y,N,"$74,104.00",0,CHGOFF,"$16,000.00","$8,000.00"
WEST MIFFLIN,PA,15122,CITIZENS BANK NATL ASSOC,RI,236115,2,1,1,0,1,1,1,Y,N,"$76,955.00",0,CHGOFF,"$35,000.00","$17,500.00"
SAN DIEGO,CA,92126,MUFG UNION BANK NATL ASSOC,CA,442110,5,6,1,0,6,1,1,Missing,N,"$24,985.00",0,CHGOFF,"$25,000.00","$12,500.00"
EDMONDS,WA,98026,BANK OF AMERICA NATL ASSOC,NC,333912,32,1,1,0,1,0,1,Y,N,"$7,265.00",0,CHGOFF,"$5,000.00","$2,500.00"
PHILADELPHIA,PA,19120,"PNC BANK, NATIONAL ASSOCIATION",DE,812320,58,1,1,1,1,1,1,Y,N,"$9,972.00",0,CHGOFF,"$10,000.00","$5,000.00"
SAINT LOUIS,MO,63139,U.S. BANK NATIONAL ASSOCIATION,OH,442299,29,4,1,0,4,0,1,Y,N,"$53,654.00",0,CHGOFF,"$20,000.00","$10,000.00"
LAFAYETTE,LA,70507,HOME BANK,LA,621320,82,5,2,0,0,1,1,N,Y,"$91,500.00",0,CHGOFF,"$91,500.00","$77,775.00"
VALENCIA,CA,91402,BANK OF AMERICA NATL ASSOC,NC,337110,49,3,1,10,3,1,1,T,N,"$20,000.00",0,CHGOFF,"$20,000.00","$10,000.00"
BROOKLYN,NY,11237,JPMORGAN CHASE BANK NATL ASSOC,IL,561510,29,6,1,1,6,0,1,N,N,"$40,400.00",0,CHGOFF,"$40,400.00","$20,200.00"
Richmond,VA,23238,SUNTRUST BANK,GA,812199,32,4,2,4,0,0,1,Y,N,"$47,499.00",0,CHGOFF,"$25,000.00","$12,500.00"


## **Performing target encoding on the following categorical columns:**
- ZIP
- City
- State

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/target-encoding.html


In [219]:
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [220]:
df_h = dataframe1
dataframe1['MIS_Status']=dataframe1['MIS_Status'].asfactor()

In [221]:
response = 'MIS_Status'
train, test = dataframe1.split_frame(ratios=[.8], seed=1234)

In [225]:
encoded_columns = ['Zip', 'City', 'State']
fold_column = "kfold_column"
train[fold_column] = train.kfold_column(n_folds=5)
df_h_f = H2OTargetEncoderEstimator(fold_column = fold_column, blending=True)
df_h_f.train(x = encoded_columns, y=response, training_frame = train)
test_df = df_h_f.transform(frame = train, as_training =True)
train_df = df_h_f.transform(frame = test, noise = 0)
gbm_with_te=H2OGradientBoostingEstimator(fold_column=fold_column,model_id="gbm_with_te")
df_h_f

targetencoder Model Build progress: |████████████████████████████████████████████| (done) 100%


original_names,encoded_column_names
City,City_te
State,State_te


In [227]:
h2o.shutdown()

H2O session _sid_9864 closed.
